# 

In [1]:
import psycopg2
import pandas as pd

## Functions to interact with database

In [89]:
def create_database():
    '''Connect to default db'''
    conn = psycopg2.connect('host=localhost dbname=postgres user=postgres password=postgres')
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # Create sparkify db with UTF8 encoding
    cur.execute('DROP DATABASE IF EXISTS accounts')
    cur.execute('CREATE DATABASE accounts')

    # Close connection to default db
    conn.close()

    # Connect to new db
    conn = psycopg2.connect('host=localhost dbname=postgres user=postgres password=postgres')
    cur = conn.cursor()

    return cur, conn

In [3]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

## Reading "Accounts Country" .csv file

In [5]:
dfAccountsCountry = pd.read_csv('../datasets/Wealth-AccountsCountry.csv')

In [63]:
#dfAccountsCountry.head()

In [25]:
dfAccountsCountry_clean = dfAccountsCountry[['Code', 'Short Name', 'Table Name', 'Long Name', 'Currency Unit']]

In [26]:
dfAccountsCountry_clean.head()

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro


## Reading "Accounts Data" .csv file

In [72]:
dfAccountData = pd.read_csv('../datasets/Wealth-AccountData.csv')

In [65]:
# dfAccountData.head()

In [66]:
# dfAccountData.columns

In [73]:
dfAccountData = dfAccountData.drop(['YOLO'], axis=1)  # axis=1 for column; axis=0 for index (raw); default is 0.
dfAccountData.head()

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3355.549,2868.997,3013.391,3259.893,3424.007,...,2.825336e+03,2.889150e+03,2.848523e+03,2.930234e+03,2.954273e+03,2.957420e+03,2.987545e+03,3072.504,3178.417,3312.927
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6353.737,5466.692,5901.072,6492.73,6872.019,...,1.525319e+04,1.578992e+04,1.579854e+04,1.647360e+04,1.681013e+04,1.699138e+04,1.730701e+04,17817.1,18445.94,19245.19
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4737.857,4059.321,4191.445,4536.972,4695.071,...,3.529144e+03,3.613606e+03,3.603694e+03,3.683810e+03,3.696744e+03,3.719608e+03,3.767628e+03,3872.142,3992.188,4161.137


## Reading "Account Series" .csv file

In [68]:
dfAccountSeries = pd.read_csv('../datasets/Wealth-AccountSeries.csv')
#dfAccountSeries.head()

In [69]:
dfAccountSeries = dfAccountSeries[['Code', 'Topic', 'Indicator Name', 'Long definition']]
dfAccountSeries.head()

,Code,Topic,Indicator Name,Long definition
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),Human capital is computed as the present value...
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con...",Human capital is computed as the present value...
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const...",Human capital is computed as the present value...
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...


## Creating tables

In [90]:
cur, conn = create_database()

In [91]:
songplay_table_create = '''
    CREATE TABLE IF NOT EXISTS AccountCountry(
        country_code VARCHAR PRIMARY KEY,
        short_name VARCHAR,
        table_name VARCHAR,
        long_name VARCHAR,
        currency_unit VARCHAR
    )
'''
cur.execute(songplay_table_create)
conn.commit()

In [92]:
# ...